Assignment - 2

Word Count

In [33]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import nltk
from nltk.corpus import stopwords
import shutil
import os
import string

#nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

output_path_1 = "output_1"
output_path_2 = "output_2"

# Remove the output directories if they already exist
if os.path.exists(output_path_1):
    shutil.rmtree(output_path_1)
if os.path.exists(output_path_2):
    shutil.rmtree(output_path_2)

try:
    # Initialize SparkSession
    spark = SparkSession.builder.appName("WordCountApp").master("local[*]").getOrCreate()

    # Get the SparkContext
    sc = spark.sparkContext

except Exception as e:
    print(f"An error occurred: {e}")





24/10/28 12:07:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [34]:
try:
    text_file = sc.textFile("littlewoman.txt")
    text_file_2 = sc.textFile("pride_and_prejudice.txt")

    # Perform transformations and actions
    counts = text_file.flatMap(lambda line: line.translate(str.maketrans("", "", string.punctuation)).lower().split()) \
                      .filter(lambda word: word not in stop_words) \
                      .map(lambda word: (word, 1)) \
                      .reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1],ascending=False)

    counts1 = text_file_2.flatMap(lambda line: line.translate(str.maketrans("", "", string.punctuation)).lower().split()) \
                         .filter(lambda word: word not in stop_words) \
                         .map(lambda word: (word, 1)) \
                         .reduceByKey(lambda a, b: a + b).sortBy(lambda x: x[1],ascending=False)

    # Save output as directories
    counts.coalesce(1).saveAsTextFile(output_path_1)
    counts1.coalesce(1).saveAsTextFile(output_path_2)

    output1 = counts.collect()
    output2 = counts.collect()

    with open('output_1_n.txt','w') as f:
        for (word, count) in output1:
            f.write(f"{word}: {count}\n")

    with open('output_2_n.txt','w') as f:
        for (word, count) in output2:
            f.write(f"{word}: {count}\n")

    # Merge partition files into a single output file
    with open("output_1.txt", "w") as outfile:
        for filename in sorted(os.listdir(output_path_1)):
            if filename.startswith("part-"):
                with open(os.path.join(output_path_1, filename), "r") as infile:
                    outfile.write(infile.read())
                    
    with open("output_2.txt", "w") as outfile:
        for filename in sorted(os.listdir(output_path_2)):
            if filename.startswith("part-"):
                with open(os.path.join(output_path_2, filename), "r") as infile:
                    outfile.write(infile.read())
    
    

    print("Word count files saved successfully.")
except Exception as e:
     print(f"An error occurred: {e}")

    

Word count files saved successfully.


Implement Djikstras

In [35]:
from pyspark.sql import SparkSession
import sys

# Initialize Spark session
spark = SparkSession.builder.appName("DijkstraShortestPath").master("local[*]").getOrCreate()
sc = spark.sparkContext

# Define a large number to represent infinity
INFINITY = float('inf')

# Load graph edges from both files
edges1 = sc.textFile("question2_1.txt")
edges2 = sc.textFile("question2_2.txt")

# Parse each line into (source, destination, weight)
edges = edges1.union(edges2).map(lambda line: line.split()).map(lambda parts: (parts[0], parts[1], float(parts[2])))

# Specify the starting node
start_node = 'A'  # Replace 'A' with the desired starting node

# Initialize distances RDD with start node distance 0, others as infinity
distances = edges.flatMap(lambda x: [(x[0], INFINITY), (x[1], INFINITY)]) \
                 .distinct() \
                 .map(lambda x: (x[0], 0 if x[0] == start_node else INFINITY))

# Initialize edges RDD for graph representation
graph = edges.map(lambda x: (x[0], (x[1], x[2])))

# Iteratively update distances using Dijkstra's algorithm
def dijkstra_update(distances, graph):
    updated_distances = distances.join(graph) \
        .flatMap(lambda x: [(x[0], x[1][0]), (x[1][1][0], x[1][0] + x[1][1][1])]) \
        .reduceByKey(lambda a, b: min(a, b))
    return updated_distances

for _ in range(10):  # Limit iterations for convergence; adjust as necessary
    distances = dijkstra_update(distances, graph)

# Collect and save results to output_2.txt
shortest_paths = distances.collect()
with open("output_3.txt", "w") as f:
    for node, dist in shortest_paths:
        f.write(f"{node}: {dist}\n")

# Find nodes with the greatest and least distance
max_node = max(shortest_paths, key=lambda x: x[1])
min_node = min(shortest_paths, key=lambda x: x[1] if x[1] > 0 else sys.maxsize)

print(f"Node with greatest distance: {max_node}")
print(f"Node with least distance: {min_node}")

# Stop Spark session



24/10/28 12:07:45 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Node with greatest distance: ('1,', inf)
Node with least distance: ('1,', inf)


Page Rank

In [37]:
import re
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder.appName("PageRank").master(["local[*]"]).getOrCreate()
sc = spark.sparkContext

# Load and parse the network file
lines = sc.textFile("question3.txt")

# Parse each line into (page, neighbors) pairs
def parse_neighbors(line):
    parts = re.split(r':\s*\[|\]', line)
    if len(parts) < 2:
        return None
    page = parts[0].strip()
    neighbors = parts[1].strip().split(', ')
    return page, neighbors

# Create an RDD of (page, list of neighbors)
links = lines.map(parse_neighbors).filter(lambda x: x is not None)

# Initialize each page's rank to 1.0
ranks = links.mapValues(lambda _: 1.0)

# Number of iterations for convergence
iterations = 10
damping_factor = 0.85  # Damping factor for PageRank

# Run PageRank algorithm for a fixed number of iterations
for _ in range(iterations):
    # Calculate contributions for each page
    contributions = links.join(ranks).flatMap(
        lambda page_neighbors_rank: [(neighbor, page_neighbors_rank[1][1] / len(page_neighbors_rank[1][0])) 
                                     for neighbor in page_neighbors_rank[1][0]]
    )
    
    # Calculate new ranks based on contributions
    ranks = contributions.reduceByKey(lambda a, b: a + b).mapValues(
        lambda rank: (1 - damping_factor) + damping_factor * rank
    )

# Collect and save the final ranks to output file
page_ranks = ranks.collect()
with open("output_page_ranks.txt", "w") as f:
    for page, rank in page_ranks:
        f.write(f"{page}: {rank}\n")

# Find the page with the highest and lowest PageRank
max_page = max(page_ranks, key=lambda x: x[1])
min_page = min(page_ranks, key=lambda x: x[1])

print(f"Page with highest rank: {max_page}")
print(f"Page with lowest rank: {min_page}")



24/10/28 12:14:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Page with highest rank: ('50', 4.176179975971073)
Page with lowest rank: ('35', 0.2593061945644343)
